In [106]:
import pandas as pd
from h3 import h3
import datetime
import folium

In [103]:
class GeoPoint:
    def __init__(self, lat, lon):
        self.__lat = lat
        self.__lon = lon
        
    @property
    def lat(self):
        return self.__lat
    
    @property
    def lon(self):
        return self.__lon

    def __str__(self):
        return "({},{})".format(self.lat, self.lon)
        
    def __repr__(self):
        return "Point({}, {})".format(self.lat, self.lon)

    def __eq__(self, other):
        return self.lat == other.lat and self.lon == other.lon
    
    def __iter__(self):
        return iter((self.lat, self.lon))
    
    def __len__(self):
        return len([v for v in self])
    
class ChargeEvent(GeoPoint):
    def __init__(self, lat, lon, hour, energy):
        super().__init__(lat, lon)
        self.hour = hour
        self.energy = energy
        
class HexNode:
    """
    max_power: The max power that a charging station can accomadate.
    plug_count: User specified plug count for a single charge station.
    """

    _MAX_RESOLUTION = 15
    _K = 7

    def __init__(self, h3_addr, resolution, max_power=150, plug_count=10):
        self._h3_addr = h3_addr
        self._resolution = resolution
        self._max_power = max_power
        self._plug_count = plug_count
        self._energy_limit = max_power * plug_count
        self._children = [None] * self._K
        self._pts = []

    @property
    def points(self):
        return self._pts
    
    @property
    def children(self):
        return self._children

    @property
    def h3_addr(self):
        return self._h3_addr
    
    def size(self):
        if self.is_leaf():
            return len(self._pts)
        else:
            return sum(c.size() for c in self._children)    
        
    def empty(self):
        # Should probably do some bookkeeping to track this value
        # instead of calling size()
        return self.size() == 0
        
    def depth(self):
        if self.is_leaf():
            return 0
        return 1 + max(c.depth() for c in self._children if c is not None)
    
    def is_leaf(self):
        return all(c is None for c in self._children)    
    
    def belongs(self, pt):
        pt_h3 = h3.geo_to_h3(pt.lat, pt.lon, self._resolution)
        if pt_h3 == self._h3_addr:
            return True

        return False
    
    def _max_events(self):
        max_events = 0
        for hr in range(0,24):
            events = 0
            for pt in self._pts:
                if pt.hour == hr:
                    events += 1
            if events > max_events:
                max_events = events
        return max_events
    
    def _max_energy(self):
        max_energy = 0
        for hr in range(0,24):
            energy = 0
            for pt in self._pts:
                if pt.hour == hr:
                    energy += pt.energy
            if energy > max_energy:
                max_energy = energy
        return max_energy

    def insert(self, p):
        return self._insert(p, self._resolution)
        
    def _insert(self, p, resolution):
        
        # Ignore objects that do not belong in this quad tree
        if not self.belongs(p):
            return False # object cannot be added
        
        if self.is_leaf() and self._max_events() <= self._plug_count:
            if resolution < self._MAX_RESOLUTION:
                self._pts.append(p)
                return True
            else:
                raise RuntimeError("Exceeded tree's max resolution: {}".format(self._MAX_RESOLUTION))
                assert False

        # Otherwise, subdivide and then add the point to whichever node will accept it
        if self.is_leaf():
            self._subdivide()
        
        for i in range(0, self._K):
            if self._children[i]._insert(p, resolution + 1): return True  
            
        print(h3.geo_to_h3(p.lat, p.lon, self._resolution+1))
        for i in range(0, self._K):
            print(self._children[i]._h3_addr)

        # Otherwise, the point cannot be inserted for some unknown reason (this should never happen)
        assert False, "Should not reach this point."
        return False
        
#     def remove(self, pt):
#         """Remove point from tree. Undo splits when possible."""
#         if not self._region.contains(pt):
#             return False
#         if self.is_leaf():
#             try:
#                 self._pts.remove(pt)
#                 return True
#             except ValueError:
#                 return False                    
#         else:
#             for child in self._children:
#                 if child.remove(pt):
#                     # Undivide if all child are empty
#                     if all(c.empty() for c in self._children):
#                         self._children = [None] * self.K
#                     # Undivide if parent can hold all points
#                     if self.size() <= self.node_capacity:
#                         self._pts = self.all_points_in_tree()
#                         self._children = [None] * self.K
#                     return True
#             return False

#     def all_points_in_tree(self):
#         result = []
#         if self.is_leaf():
#             return [Point(p.x, p.y) for p in self._pts]
        
#         for qt in self._children:
#             if qt is not None:
#                 result.extend(qt.all_points_in_tree())    
                
#         return result
        
#     def find_points_in_region(self, region):
#         result = []

#         # Automatically abort if the range does not intersect this quad
#         if not self._region.intersects(region):
#             return result # Empty list
        
#         # Terminate here, if there are no children
#         if self.is_leaf():
#             return [p for p in self._pts if region.contains(p)]

#         # Otherwise, add the points from the children
#         for qt in self._children:
#             if qt is not None:
#                 result.extend(qt.find_points_in_region(region))

#         return result

    def _subdivide(self):
        print("subdividing..")
        children_h3 = h3.h3_to_children(self.h3_addr, self._resolution + 1)
        
        for i, child_h3 in enumerate(children_h3):
            self._children[i] = HexNode(child_h3, self._resolution+1, self._max_power, self._plug_count)

        # Distribute points
        for p in self._pts:
            for hx in self._children:
                if hx.insert(p):
                    break

        self._pts = []

In [7]:
CHARGE_POWER = 150 #kw

In [8]:
df = pd.read_csv('stlog.csv')
df['start_time'] = df['start_time'].map(lambda x: datetime.datetime.utcfromtimestamp(x))
df['end_time'] = df['end_time'].map(lambda x: datetime.datetime.utcfromtimestamp(x))
df['t_delta'] = df['end_time'] - df['start_time']
df['hour'] = df['start_time'].map(lambda x: x.hour)
df['charge_energy_kwh'] = df['t_delta'].map(lambda x: x.total_seconds()*CHARGE_POWER/3600)

In [9]:
df.head()

,station_id,veh_id,start_time,end_time,soc_i,soc_f,lat,lon,charge_time,t_delta,hour,charge_energy_kwh
0,8382,10,2017-02-02 02:16:29.275159,2017-02-02 02:42:07.275159,0.193133,0.8,30.285866,-97.731067,1538,00:25:38,2,64.083333
1,8893,50,2017-02-02 02:14:54.709874,2017-02-02 02:40:38.709874,0.190583,0.8,30.265866,-97.736067,1544,00:25:44,2,64.333333
2,8115,76,2017-02-02 02:21:57.543537,2017-02-02 02:47:49.543537,0.186979,0.8,30.295866,-97.786067,1552,00:25:52,2,64.666667
3,8763,153,2017-02-02 02:24:34.611807,2017-02-02 02:50:03.611807,0.197234,0.8,30.270866,-97.746067,1529,00:25:29,2,63.708333
4,7740,36,2017-02-02 02:32:14.110785,2017-02-02 02:58:22.110785,0.179911,0.8,30.310866,-97.741067,1568,00:26:08,2,65.333333


In [62]:
df.groupby('hour').charge_energy_kwh.count()

hour
0     11
1     15
2     29
3     32
4     31
5     36
6     36
7     26
8     27
9     21
10     8
11     9
12     5
13     5
14     8
15     7
16     8
17    17
18    16
19    20
20    15
21    18
22    13
23    20
Name: charge_energy_kwh, dtype: int64

In [33]:
h3_address = h3.geo_to_h3(df.lat.iloc[200], df.lon.iloc[200], 5)
print(h3_address)

85489e37fffffff


In [104]:
hex_node = HexNode(h3_address, 5)

In [105]:
for i, row in df.iterrows():
    ce = ChargeEvent(row.lat, row.lon, row.hour, row.charge_energy_kwh)
    hex_node.insert(ce)

subdividing..
86489e267ffffff
86489e377ffffff
86489e357ffffff
86489e36fffffff
86489e34fffffff
86489e367ffffff
86489e347ffffff
86489e35fffffff


AssertionError: Should not reach this point.

In [86]:
for child in hex_node.children:
    print(child.children)

[None, None, None, None, None, None, None]
[None, None, None, None, None, None, None]
[None, None, None, None, None, None, None]
[None, None, None, None, None, None, None]
[None, None, None, None, None, None, None]
[None, None, None, None, None, None, None]
[None, None, None, None, None, None, None]


In [87]:
hex_node.children